## 0 Global settings

In [1]:
# Import the needed credential and management objects from the libraries.
import sys
import os

from azure.identity import InteractiveBrowserCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace, Data, AmlCompute 
from azure.ai.ml.constants import AssetTypes
from dotenv import load_dotenv, version

print(f"System version: {sys.version}")
print("Setup Complete")

System version: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Setup Complete


## 1 Create a Resource Group

In [2]:
# Retrieves environment variables from .env.
load_dotenv()

subscription_id = os.getenv("SUBSCRIPTION_ID")  # preferably using env variable
resource_group = os.getenv("RESOURCE_GROUP")  # preferably using env variable
workspace_name = os.getenv("WORKSPACE_NAME")  # preferably using env variable
location = os.getenv("LOCATION")  # preferably using env variable

In [3]:
# Acquire a credential object using InteractiveBrowserCredential.
credential = InteractiveBrowserCredential()

In [4]:
# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)

In [5]:
# Provision the resource group.
rg_result = resource_client.resource_groups.create_or_update(
    resource_group, {"location": location}
)

print(
    "Provisioned resource group successful"
)


Provisioned resource group successful


## 2 Create an Azure Machine Learning workspace

In [6]:
# Get a handle to the subscription
ml_client = MLClient(credential, subscription_id, resource_group)

ws = Workspace(
    name=workspace_name,
    location=location,
    display_name="MLW Stories For Every Start",
    description="Azure machine learning workspace for recommender system",
)
ws = ml_client.workspaces.begin_create(ws).result()

print(
    "Azure Machine Learning Workspace provisioned successfully"
)

The deployment request mlw-storiesforeverystar-dev-1684543 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F578e24c5-0cd4-4c93-9751-a09f28da61be%2FresourceGroups%2Frg-storiesforeverystar-shared-002%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Fmlw-storiesforeverystar-dev-1684543
Creating Storage Account: (mlwstoristorageb5d8fabd1  ) ...  Done (24s)
Creating Key Vault: (mlwstorikeyvault0502dce6  )  Done (20s)
Creating AzureML Workspace: (mlw-storiesforeverystar-dev  ) ..  Done (22s)
Total time : 48s



Azure Machine Learning Workspace provisioned successfully


## 3 Create data assets

In [7]:
# Get a handle to workspace
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [8]:
# List the datastores
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

workspaceworkingdirectory
workspacefilestore
workspaceblobstore
workspaceartifactstore


In [9]:
data_path = '../data/processed/user-item-ratings.csv'

my_data = Data(
    path=data_path,
    type=AssetTypes.URI_FILE,
    description="Contains ratings given to children's books in the public domain",
    name="user-item-ratings"
)

ml_client.data.create_or_update(my_data)
print(my_data)

Uploading user-item-ratings.csv (< 1 MB): 100%|##########| 1.29M/1.29M [00:02<00:00, 545kB/s]




description: Contains ratings given to children's books in the public domain
name: user-item-ratings
path: azureml://datastores/workspaceblobstore/paths/LocalUpload/e8387d8de409093813b725147f48c54a/user-item-ratings.csv
properties: {}
tags: {}
type: uri_file



## 4 Create a compute cluster

In [10]:
# Name assigned to the compute cluster
cpu_compute_target = "aml-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS11_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=2,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=120,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)


Creating a new cpu compute target...
